1. Project Title & Team Information

Title:  Accident Risk Analysis: Urban vs. Rural Populations

Team Members: Dave Woodford, Tim Mai, and Dave Woodford

#TO DO: Team Members Table (Name, Role, Skills, Contribution)

2. Project Definition & Motivation

Objective: "Does accident frequency increase in high-population counties? Do factors differ between rural and metro areas?"

Stakeholders: Dept of Transporation, Railroad Administration, Railroad companies, local municipaliteis, and the public

Potential Application: A risk-assessment tool or "safe travel" dashboard


## 3. Data Sources & Selection

Source A: Rail Equipment Accident Data (Form 54).
    Justification: Primary source for accident frequency and cause codes.
    Link: https://data.transportation.gov/Railroads/Rail-Equipment-Accident-Incident-Data-Form-54-/85tf-25kj/about_data

Source B: USDA Rural-Urban Continuum Codes (2023).
    Justification: Provides the necessary demographic context (Rural vs. Metro) to normalize accident data.
    Link: https://www.ers.usda.gov/data-products/rural-urban-continuum-codes

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme(style="whitegrid")

rail_accident_path = "../data/raw/Rail_Equipment_Accident_Incident_Data_(Form_54)_20260114.csv.zip"
population_path = "../data/raw/Ruralurbancontinuumcodes2023.csv"

df_accident_all = pd.read_csv(
    rail_accident_path,
    low_memory=False
)

df_population = pd.read_csv(
    population_path,
    encoding="latin1"
)

print(f"Rail Data Loaded. Shape: {df_accident_all.shape}")
print(f"Population Data Loaded. Shape: {df_population.shape}")
print()
display(df_accident_all.head(3))
print()
display(df_population.head(3))

Rail Data Loaded. Shape: (223529, 155)
Population Data Loaded. Shape: (9703, 5)



,Reporting Railroad Code,Reporting Railroad Name,Year,Accident Number,PDF Link,Accident Year,Accident Month,Other Railroad Code,Other Railroad Name,Other Accident Number,...,Reporting Parent Railroad Name,Reporting Railroad Holding Company,Reporting Railroad Individual Class,Reporting Railroad Passenger,Reporting Railroad Commuter,Reporting Railroad Switching Terminal,Reporting Railroad Tourist,Reporting Railroad Freight,Reporting Railroad Short Line,Location
0,SOO,SOO Line Railroad Company,1981,CA28,https://safetydata.fra.dot.gov/Officeofsafety/...,81,8,NaN,NaN,NaN,...,CANADIAN PACIFIC KANSAS CITY,Not Assigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,NaN
1,MP,Missouri Pacific Railroad Company,1981,81230,https://safetydata.fra.dot.gov/Officeofsafety/...,81,9,NaN,NaN,NaN,...,Union Pacific Railroad Company,Union Pacific Railroad Company,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,NaN
2,SP,Southern Pacific Transportation Company,1980,W1200,https://safetydata.fra.dot.gov/Officeofsafety/...,80,1,NaN,NaN,NaN,...,Union Pacific Railroad Company,Union Pacific Railroad Company,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,Not Assigned,NaN


,FIPS,State,County_Name,Attribute,Value
0,1001,AL,Autauga County,Population_2020,58805
1,1001,AL,Autauga County,RUCC_2023,2
2,1001,AL,Autauga County,Description,"Metro - Counties in metro areas of 250,000 to ..."


## 4. Methodlogy: Data Cleaning

- Explain: "We padded State (2 digits) and County (3 digits) to create a valid 5-digit FIPS key for merging."

In [8]:
df_accident = df_accident_all.copy()

# Clean accident data
df_accident["State Code FIPS"] = df_accident["State Code"].fillna(0).astype(int).astype(str).str.zfill(2)
df_accident["County Code FIPS"] = df_accident["County Code"].fillna(0).astype(int).astype(str).str.zfill(3)
df_accident["FIPS"] = (df_accident["State Code FIPS"] + df_accident["County Code FIPS"]).astype(int)

# Clean population data
df_population_wide = df_population.pivot(index="FIPS", columns="Attribute", values="Value").reset_index()
df_population_wide["FIPS"] = df_population_wide["FIPS"].astype(int)
df_population_wide.columns.name = None



##  5. Methodology: The Merge

In [7]:
# Merge
df_merged = pd.merge(
    df_accident,
    df_population_wide,
    on="FIPS",
    how="left"
)

# Checks
print(f"Merged Shape: {df_merged.shape}")
print("New columns available:", list(df_population_wide.columns))

cols_to_show = ["FIPS", "State Name", "Description", "Population_2020"]
display(df_merged[cols_to_show].head())

Merged Shape: (223529, 161)
New columns available: ['FIPS', 'Description', 'Population_2020', 'RUCC_2023']


,FIPS,State Name,Description,Population_2020
0,27121,MINNESOTA,"Nonmetro - Urban population of fewer than 5,00...",11308
1,31025,NEBRASKA,"Metro - Counties in metro areas of 250,000 to ...",26598
2,48157,TEXAS,Metro - Counties in metro areas of 1 million p...,822779
3,2000,ALASKA,NaN,NaN
4,20177,KANSAS,Metro - Counties in metro areas of fewer than ...,178909


6. Initial Exploratory Analysis (EDA)

The Question: "Is there a correlation between population size and raw accident count?"

The Visualization: Your scatter plot (plt.scatter(x,y)).

The Caption/Interpretation: "The plot shows a positive correlation (0.58), but outliers exist. This suggests that while population is a factor, other variables (track density, traffic volume) likely play a role."

7. Limitations and Future Improvements

Current Limitations: The analysis currently looks at raw counts, not normalized by "train miles" or "track miles" (traffic density).

Missing Variables: Weather, Time of Day, Seasonality, etc. 

8. Dissemination & Next Steps

Target Audience: 
    US Population (safety consciousness)
    Gov Agencies (maintenance and policy prioritization)
    Railroad companies

Next Steps:
    Analyzing specific "Cause Codes" (Human error vs. Equipment failure) across those bins.